<a href="https://colab.research.google.com/github/the-khiem7/AWS.FirstCloudJourney/blob/main/SnakeTraining_V6_YOLOv12_Khiem_Bbox5000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment setup

### Nvidia profile

In [ ]:
!nvidia-smi

### Configure model name

In [ ]:
import os
from datetime import datetime, timedelta, timezone

HOME = os.getcwd()
print(f"📂 HOME: {HOME}")

PROJECT_NAME = "SnakeTraining"
VERSION      = "V6"
MODEL_ARCH   = "YOLOv12"
AUTHOR       = "Khiem"
DATA_INFO    = "Bbox5291Complete"

# Lấy thời gian hiện tại theo múi giờ UTC+7
utc_plus_7 = timezone(timedelta(hours=7))
current_time = datetime.now(utc_plus_7).strftime("%Y%m%d_%H%M")

# Ghép chuỗi tên model kèm timestamp
notebook_name = f"{PROJECT_NAME}_{VERSION}_{MODEL_ARCH}_{AUTHOR}_{DATA_INFO}_40epoch_10patience_{current_time}"

print(f"📘 Configured Model Name: {notebook_name}")

### Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Define Dataset Paths and copy to Colab local Storage

In [ ]:
import os
import shutil
import yaml
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor

# --- CONFIG ---
DRIVE_DATASET_PATH = '/content/drive/MyDrive/SnakeDataset/SnakeAid-YOLOv12-5291BBox'
DATASET_PATH = '/content/snake_dataset'
DATA_YAML_PATH = f'{DATASET_PATH}/data.yaml'

def check_dataset_integrity(path):
    """Kiểm tra thông minh: Trả về True nếu data đã đầy đủ"""
    # 1. Check file cấu hình
    if not os.path.exists(f"{path}/data.yaml"):
        return False

    # 2. Check các folder quan trọng (train, valid, test)
    # Nếu bất kỳ folder nào thiếu hoặc rỗng -> coi như lỗi -> copy lại
    required_dirs = [
        'train/images', 'train/labels',
        'valid/images', 'valid/labels',
        'test/images', 'test/labels'
    ]

    for d in required_dirs:
        dir_path = os.path.join(path, d)
        if not os.path.exists(dir_path):
            return False
        if not os.listdir(dir_path): # Folder rỗng
            return False

    return True

def copy_file_worker(args):
    """Hàm worker để copy 1 file (dùng cho đa luồng)"""
    src, dst = args
    try:
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy2(src, dst)
    except Exception as e:
        # Có thể log error nếu cần
        pass

def fast_copy_multithread(src_root, dst_root, workers=16):
    """Copy dữ liệu sử dụng đa luồng để tối ưu I/O"""
    if os.path.exists(dst_root):
        print(f"🗑️ Phát hiện dữ liệu không toàn vẹn. Đang xóa {dst_root}...")
        shutil.rmtree(dst_root)

    print("🔍 Đang quét danh sách file nguồn (Scanning)...")
    files_to_copy = []
    for root, dirs, files in os.walk(src_root):
        for file in files:
            src_file = os.path.join(root, file)
            rel_path = os.path.relpath(src_file, src_root)
            dst_file = os.path.join(dst_root, rel_path)
            files_to_copy.append((src_file, dst_file))

    print(f"🚀 Tìm thấy {len(files_to_copy)} files. Bắt đầu copy tốc độ cao (16 threads)...")

    # ThreadPoolExecutor giúp request nhiều file cùng lúc từ Drive
    with ThreadPoolExecutor(max_workers=workers) as executor:
        list(tqdm(executor.map(copy_file_worker, files_to_copy), total=len(files_to_copy), unit="file"))

# --- MAIN FLOW ---
if check_dataset_integrity(DATASET_PATH):
    print("✅ Smart Check: Dữ liệu đã đầy đủ và hợp lệ tại Local. (Skipped Copy)")
else:
    print(f"⚠️ Smart Check: Dữ liệu thiếu hoặc chưa có. Tiến hành copy...")
    try:
        fast_copy_multithread(DRIVE_DATASET_PATH, DATASET_PATH)

        # Cập nhật đường dẫn trong data.yaml
        if os.path.exists(DATA_YAML_PATH):
            with open(DATA_YAML_PATH, 'r') as f:
                data_config = yaml.safe_load(f)

            data_config['path'] = DATASET_PATH
            # Fix đường dẫn relative
            for key in ['train', 'val', 'test']:
                if key in data_config and isinstance(data_config[key], str):
                    if 'drive' in data_config[key].lower():
                        folder = 'valid' if key == 'val' else key
                        data_config[key] = f'{folder}/images'

            with open(DATA_YAML_PATH, 'w') as f:
                yaml.dump(data_config, f)
            print("✅ Cấu hình data.yaml hoàn tất!")
        else:
            print("⚠️ Không tìm thấy data.yaml sau khi copy!")

    except Exception as e:
        print(f"❌ Lỗi khi copy: {e}")

### Dataset file validation

In [ ]:
import os

def check_dataset(root, folders=("train", "valid", "test")):
    table = Table(
        title="Dataset Structure",
        box=box.SIMPLE_HEAD,
        show_lines=False,
        pad_edge=False
    )

    table.add_column("Component", style="bold cyan", justify="center")
    table.add_column("Status/Images", justify="center")
    table.add_column("Labels", justify="center")

    missing_files = False

    yaml_path = os.path.join(root, "data.yaml")
    yaml_exists = os.path.exists(yaml_path)

    if not yaml_exists:
        missing_files = True

    table.add_row(
        "data.yaml",
        "[green]FOUND" if yaml_exists else "[red]MISSING",
        "-"
    )

    for folder in folders:
        fp = os.path.join(root, folder)
        img = os.path.join(fp, "images")
        lbl = os.path.join(fp, "labels")

        img_exists = os.path.exists(img)
        lbl_exists = os.path.exists(lbl)

        if not img_exists or not lbl_exists:
            missing_files = True

        table.add_row(
            folder.upper(),
            "[green]FOUND" if img_exists else "[red]MISSING",
            "[green]FOUND" if lbl_exists else "[red]MISSING",
        )

    panel = Panel.fit(
        table,
        title="🐍 Snake Dataset Check",
        border_style="cyan",
        box=box.SIMPLE
    )

    print(panel)

    if missing_files:
        raise FileNotFoundError("Dataset structure is incomplete! Please check your dataset path and structure.")

check_dataset(DATASET_PATH)

### Label distribution per class

In [ ]:
import yaml
import glob
import os
import matplotlib.pyplot as plt
from collections import Counter
from tqdm.notebook import tqdm

# --- 1. Load Class Names from data.yaml ---
with open(DATA_YAML_PATH, 'r') as f:
    data_cfg = yaml.safe_load(f)

# Handle 'names' format (dict or list)
if isinstance(data_cfg['names'], dict):
    id2name = data_cfg['names']
else:
    id2name = {i: name for i, name in enumerate(data_cfg['names'])}

# --- 2. Count Objects per Class ---
# Quét tất cả file nhãn trong train, valid, test
label_files = []
for split in ['train', 'valid', 'test']:
    path = os.path.join(DATASET_PATH, split, 'labels')
    if os.path.exists(path):
        label_files.extend(glob.glob(os.path.join(path, "*.txt")))

print(f"🔍 Found {len(label_files)} label files in {DATASET_PATH}")

cls_counts = Counter()
for lf in tqdm(label_files, desc="Counting labels"):
    with open(lf, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if parts:
                cls_id = int(parts[0])
                cls_counts[cls_id] += 1

# --- 3. Prepare Data for Plotting ---
data = []
# Sort by Class ID to ensure consistent order
for cid in sorted(id2name.keys()):
    cname = id2name[cid]
    cnt = cls_counts.get(cid, 0)
    data.append((cname, cnt))

# Unpack
class_names, class_counts = zip(*data)

# --- 4. Plotting (Adapted from Template) ---
plt.figure(figsize=(20, 10))  # Tăng chiều cao một chút
bars = plt.bar(class_names, class_counts, color='skyblue', edgecolor='navy', alpha=0.8)

# Tăng fontsize cho xticks (tên class)
plt.xticks(rotation=90, ha="center", fontsize=14)
plt.xlabel("Snake Class", fontsize=14, fontweight='bold')
plt.ylabel("Number of Labels (Objects)", fontsize=14, fontweight='bold')
plt.title(f"Label Distribution per Snake Class (Total: {sum(class_counts)})", fontsize=16, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Ghi số lên đầu cột
max_h = max(class_counts) if class_counts else 0
for bar in bars:
    h = bar.get_height()
    if h > 0:
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            h + (max_h * 0.01),
            str(int(h)),
            ha="center",
            va="bottom",
            fontsize=12,
            rotation=90 if len(class_names) > 30 else 0
        )

plt.tight_layout()
plt.show()

# Install YOLOv12 and SuperVision

## Install essential python package

In [1]:
!pip install -q git+https://github.com/sunsmarterjie/yolov12.git supervision

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 12.3 MB/s eta 0:00:00


In [3]:
import ultralytics
print(ultralytics.__version__)

8.3.63


# Test the base models

### Download example data

In [ ]:
!wget https://media.roboflow.com/notebooks/examples/dog.jpeg

### Run inference

In the example, we're using the `yolov12l.pt` model, but you can experiment with different model sizes by simply swapping out the model name during initialization. Options include `yolov12n.pt`, `yolov12s.pt`, `yolov12m.pt`, `yolov12l.pt`, and `yolov12x.pt`.

In [ ]:
import cv2
from ultralytics import YOLO
import supervision as sv


image_path = f"{HOME}/dog.jpeg"
image = cv2.imread(image_path)

model = YOLO('yolov12l.pt')

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results)

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

# Fine-tune YOLOv12 model

We are now ready to fine-tune our YOLOv12 model. In the code below, we initialize the model using a starting checkpoint—here, we use `yolov12s.yaml`, but you can replace it with any other model (e.g., `yolov12n.pt`, `yolov12m.pt`, `yolov12l.pt`, or `yolov12x.pt`) based on your preference. We set the training to run for 100 epochs in this example; however, you should adjust the number of epochs along with other hyperparameters such as batch size, image size, and augmentation settings (scale, mosaic, mixup, and copy-paste) based on your hardware capabilities and dataset size.

**Note:** **Note that after training, you might encounter a `TypeError: argument of type 'PosixPath' is not iterable error` — this is a known issue, but your model weights will still be saved, so you can safely proceed to running inference.**

In [ ]:
# from ultralytics import YOLO

# model = YOLO('yolov12m.pt')
# results = model.train(data=DATA_YAML_PATH, epochs=100, patience=10, cache=True)

### GPU detect

**Imports**

In [ ]:
import os
import torch

from rich import print
from rich.panel import Panel
from rich.table import Table
from rich import box

**Helpers + Rich logger**

In [ ]:
def is_kaggle_env() -> bool:
    # Kaggle notebooks always set this
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ

def log_gpu_info():
    table = Table(
        title="GPU Runtime Information",
        box=box.SIMPLE_HEAVY,
        show_header=True,
        header_style="bold cyan"
    )
    table.add_column("Metric", style="bold")
    table.add_column("Value")

    if not torch.cuda.is_available():
        table.add_row("CUDA Available", "❌ No")
        table.add_row("GPU Count", "0")
        print(table)
        return

    idx = torch.cuda.current_device()
    props = torch.cuda.get_device_properties(idx)

    total_gb = props.total_memory / (1024**3)
    reserved_gb = torch.cuda.memory_reserved(idx) / (1024**3)
    allocated_gb = torch.cuda.memory_allocated(idx) / (1024**3)
    free_gb = total_gb - reserved_gb

    table.add_row("CUDA Available", "✅ Yes")
    table.add_row("GPU Name", props.name)
    table.add_row("GPU Count", str(torch.cuda.device_count()))
    table.add_row("VRAM Total", f"{total_gb:.2f} GB")
    table.add_row("VRAM Reserved", f"{reserved_gb:.2f} GB")
    table.add_row("VRAM Allocated", f"{allocated_gb:.2f} GB")
    table.add_row("VRAM Free (approx)", f"{free_gb:.2f} GB")

    print(table)

**Main detection (select profile)**

In [ ]:
def select_training_profile():
    kaggle = is_kaggle_env()

    env_name = "Kaggle" if kaggle else "Colab / Other"
    print(
        Panel.fit(
            f"[bold]Environment:[/bold] {env_name}",
            title="Runtime Environment",
            border_style="green"
        )
    )

    # Log GPU info (log only)
    log_gpu_info()

    if not torch.cuda.is_available():
        return "CPU"

    gpu_name = torch.cuda.get_device_name(0).lower()

    # Kaggle mapping
    if kaggle:
        if "p100" in gpu_name:
            return "KAGGLE_P100"
        if "t4" in gpu_name:
            return "KAGGLE_T4X2"
        return "KAGGLE_P100"

    # Colab mapping
    if "a100" in gpu_name:
        return "A100"
    if "l4" in gpu_name:
        return "L4"
    if "t4" in gpu_name:
        return "T4"

    return "T4"


TRAIN_PROFILE = select_training_profile()

print(
    Panel.fit(
        f"[bold cyan]Selected training profile:[/bold cyan] [bold yellow]{TRAIN_PROFILE}[/bold yellow]",
        title="Profile Selection",
        border_style="cyan"
    )
)

## Training profiles

**`epochs=200`**

* **Chức năng:** Xác định số vòng lặp huấn luyện tối đa mà mô hình sẽ chạy trên toàn bộ tập dữ liệu.
* **Lý do lựa chọn:** Với đối tượng rắn thường nhỏ và khó phân biệt với nền, các chỉ số như mAP và Recall thường cải thiện muộn, vì vậy cần nhiều epoch hơn mức mặc định (100) để mô hình hội tụ đầy đủ.

---

**`patience=40`**

* **Chức năng:** Điều khiển cơ chế early stopping, mô hình sẽ dừng huấn luyện nếu metric validation không cải thiện sau số epoch này.
* **Lý do lựa chọn:** Trong bài toán SnakeAid, metric thường dao động mạnh do vật thể nhỏ và nền phức tạp, nên đặt patience cao để tránh dừng huấn luyện quá sớm khi mô hình chưa đạt trạng thái tối ưu.

---

**`imgsz=896`**

* **Chức năng:** Thiết lập độ phân giải ảnh đầu vào cho quá trình huấn luyện.
* **Lý do lựa chọn:** Rắn trong ảnh thường chiếm rất ít pixel, tăng độ phân giải giúp mô hình giữ lại nhiều chi tiết không gian hơn, từ đó cải thiện khả năng phát hiện rắn nhỏ hoặc ở xa.

---

**`batch=16`**

* **Chức năng:** Xác định số lượng ảnh được xử lý trong một bước cập nhật gradient.
* **Lý do lựa chọn:** Batch size vừa phải giúp cân bằng giữa độ ổn định của gradient và khả năng học các vật thể nhỏ, tránh hiện tượng làm mượt quá mức khi batch quá lớn.

---

**`cache=True`**

* **Chức năng:** Lưu dữ liệu huấn luyện vào bộ nhớ đệm để tăng tốc độ đọc dữ liệu.
* **Lý do lựa chọn:** Dataset được lưu trên Google Drive khi train bằng Colab, việc bật cache giúp giảm độ trễ I/O và rút ngắn thời gian huấn luyện tổng thể.

---

**`workers=8`**

* **Chức năng:** Xác định số tiến trình CPU song song được sử dụng để đọc dữ liệu, tiền xử lý ảnh và áp dụng các augmentation trước khi dữ liệu được đưa vào GPU trong quá trình huấn luyện.
* **Lý do lựa chọn:** Với ảnh độ phân giải cao (`imgsz=896`) và dữ liệu lưu trên Google Drive khi huấn luyện bằng Colab A100, việc sử dụng `workers=8` giúp tận dụng hiệu quả tài nguyên CPU, tránh tình trạng GPU phải chờ dữ liệu và đảm bảo tốc độ huấn luyện ổn định cho bài toán nhận diện rắn nhỏ và ở xa.

---

**`cos_lr=True`**

* **Chức năng:** Sử dụng chiến lược giảm learning rate theo hàm cosine trong quá trình huấn luyện.
* **Lý do lựa chọn:** Cosine learning rate giúp fine-tune mượt hơn ở các epoch sau, giảm nguy cơ dao động mạnh và giúp mô hình ổn định khi tối ưu cho các vật thể nhỏ.

In [ ]:
CONFIGS = {

    # Quick sanity / pipeline check
    "DEBUG": dict(
        epochs=1,
        patience=0,
        imgsz=512,
        batch=2,
        nbs=1,
        cache=False,
        workers=1,
    ),

    # CPU-only
    "CPU": dict(
        epochs=120,
        patience=30,
        imgsz=640,
        batch=2,
        nbs=8,     # effective batch = 8
        cache="disk",
        workers=2,
    ),

    # Colab Free (Tesla T4 16GB VRAM)
    "T4": dict(
        epochs=80,
        patience=20,
        imgsz=768,
        batch=4,
        nbs=8,     # effective batch = 8
        cache="disk",
        workers=2,
    ),

    # Kaggle GPU P100 (16GB VRAM)
    "KAGGLE_P100": dict(
        epochs=80,
        patience=20,
        imgsz=768,
        batch=4,
        nbs=8,     # effective batch = 8
        cache="disk",
        workers=2,
    ),

    # Kaggle GPU T4 x2 (treated as single T4 unless DDP enabled)
    "KAGGLE_T4X2": dict(
        epochs=80,
        patience=20,
        imgsz=768,
        batch=4,
        nbs=8,     # effective batch = 8
        cache="disk",
        workers=2,
    ),

    # Colab Pro (NVIDIA L4 24GB VRAM)
    "L4": dict(
        epochs=60,
        patience=12,
        imgsz=896,
        # batch=8,
        batch=10,
        nbs=16,
        cache=True,
        workers=6,
    ),

    # Colab Pro (A100 40GB VRAM)
    "A100": dict(
        epochs=200,
        patience=40,
        imgsz=896,
        batch=16,
        nbs=16,
        cache=True,
        workers=8,
    ),
}

cfg = CONFIGS[TRAIN_PROFILE]

print("🧠 Loaded training configuration:")
for k, v in cfg.items():
    print(f"  {k}: {v}")

## MAIN TRAIN RUNNER

#### `mixup=0.0`

* **Chức năng:** Điều chỉnh mức độ sử dụng augmentation mixup (trộn hai ảnh và nhãn).
* **Lý do lựa chọn:** Mixup có thể tạo ra các hình ảnh không thực tế đối với đối tượng dài và mảnh như rắn, làm giảm chất lượng học, vì vậy được tắt hoàn toàn trong bài toán này.

---

#### `copy_paste=0.0`

* **Chức năng:** Điều khiển augmentation copy-paste, trong đó đối tượng được cắt và dán sang ảnh khác.
* **Lý do lựa chọn:** Copy-paste có thể tạo ra các vị trí rắn không tự nhiên trong ảnh, gây nhiễu cho mô hình, nên được tắt để giữ tính thực tế của dữ liệu huấn luyện.

In [ ]:
from ultralytics import YOLO

# Load pretrained model
model = YOLO("yolov12m.pt")

results = model.train(
    data=(DATA_YAML_PATH),
    project=f"{HOME}/runs/detect",
    name='train',
    exist_ok=True,

    # epochs=cfg["epochs"],
    # patience=cfg["patience"],
    epochs=40,
    patience=10,
    imgsz=cfg["imgsz"],
    batch=cfg["batch"],
    # nbs=cfg["nbs"],
    cache=cfg["cache"],
    workers=cfg["workers"],
    cos_lr=True,

    # Snake-specific augmentation control
    mixup=0.0,
    copy_paste=0.0,
)

print("✅ Training finished")
print(f"📁 Weights saved to: runs/detect/train/weights/")

## Save model file into google drive

In [ ]:
import shutil
import os

# Define the source path of the best trained model
source_path = f'{HOME}/runs/detect/train/weights/best.pt'

# Define the destination directory in Google Drive
destination_dir = '/content/drive/MyDrive/SnakeAIModels'
os.makedirs(destination_dir, exist_ok=True)

# Sử dụng biến 'notebook_name' đã được define ở cell Setup đầu notebook
print(f"💾 Saving model for notebook: {notebook_name}")
destination_filename = f"{notebook_name}.pt"
destination_path = os.path.join(destination_dir, destination_filename)

# Copy the model file to Google Drive
if os.path.exists(source_path):
    shutil.copy(source_path, destination_path)
    print(f"✅ Trained model saved to: {destination_path}")
else:
    print(f"❌ Source model not found at {source_path}. Make sure training has finished successfully.")

# Train Report

### Inspect Training Output Directory

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!ls {HOME}/runs/detect/train/

### Display Training Results Plot

In [ ]:
from IPython.display import Image

Image(filename=f'{HOME}/runs/detect/train/results.png', width=1000)

### Display Confusion Matrix

In [ ]:
from IPython.display import Image

Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=1000)

In [ ]:
from IPython.display import Image

Image(filename=f'{HOME}/runs/detect/train/confusion_matrix_normalized.png', width=1000)

### Load Test Dataset and Display Classes

In [ ]:
import supervision as sv

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{DATASET_PATH}/test/images",
    annotations_directory_path=f"{DATASET_PATH}/test/labels",
    data_yaml_path=DATA_YAML_PATH
)

ds.classes

### Calculate Mean Average Precision (mAP)

In [ ]:
from supervision.metrics import MeanAveragePrecision

model = YOLO(f'{HOME}/runs/detect/train/weights/best.pt')

predictions = []
targets = []

for _, image, target in ds:
    results = model(image, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    predictions.append(detections)
    targets.append(target)

map = MeanAveragePrecision().update(predictions, targets).compute()

In [ ]:
print("mAP 50:95", map.map50_95)
print("mAP 50", map.map50)
print("mAP 75", map.map75)

In [ ]:
map.plot()

### Qualitative Analysis (Ground Truth vs Predictions)

Các hình ảnh này cung cấp bằng chứng trực quan về khả năng định vị và phát hiện rắn của mô hình trên tập validation.

Việc so sánh trực tiếp giữa ground truth và prediction giúp:

* Đánh giá độ chính xác của bounding box
* Quan sát các trường hợp false positive và false negative
* Kiểm tra hiệu quả của mô hình trong các bối cảnh khó như rắn nhỏ, xa hoặc lẫn nền

Phân tích định tính này bổ sung cho các metric định lượng, giúp tăng độ tin cậy của kết quả đánh giá.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

def show_val_batch(batch_id: int):
    """
    Display Ground Truth vs Prediction for a given validation batch.
    Batch index is 0-based (e.g., batch_id=0 → val_batch0_*).
    """
    gt_path = f"{HOME}/runs/detect/train/val_batch{batch_id}_labels.jpg"
    pred_path = f"{HOME}/runs/detect/train/val_batch{batch_id}_pred.jpg"

    gt = Image.open(gt_path)
    pred = Image.open(pred_path)

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    axes[0].imshow(gt)
    axes[0].set_title(f"Ground Truth Annotations (Batch{batch_id})", fontsize=12)
    axes[0].axis("off")

    axes[1].imshow(pred)
    axes[1].set_title(f"Model Predictions (Batch{batch_id})", fontsize=12)
    axes[1].axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
show_val_batch(BATCH_HEAD)

In [ ]:
show_val_batch(BATCH_MID)

In [ ]:
show_val_batch(BATCH_TAIL)

### Training Batch Visualization

Các hình ảnh batch trong quá trình huấn luyện được sử dụng để kiểm tra trực quan chất lượng dữ liệu và tính hợp lệ của annotation.

Thông qua các batch mẫu, có thể quan sát:

* Sự chính xác của bounding box so với đối tượng thực tế
* Khả năng biểu diễn các trường hợp khó như rắn nhỏ, xa hoặc lẫn nền phức tạp
* Ảnh hưởng của các kỹ thuật augmentation đến hình dạng và ngữ cảnh của đối tượng

Các batch ở giai đoạn đầu và giai đoạn muộn của huấn luyện được trình bày nhằm xác nhận rằng dữ liệu đầu vào luôn giữ được tính nhất quán và phù hợp trong suốt quá trình training, góp phần đảm bảo độ tin cậy của kết quả mô hình.

In [ ]:
import glob
import os
import matplotlib.pyplot as plt
from PIL import Image

batch_imgs = sorted(
    glob.glob(f"{HOME}/runs/detect/train/train_batch*.jpg")
)

if len(batch_imgs) >= 4:
    selected = [
        batch_imgs[0],
        batch_imgs[1],
        batch_imgs[len(batch_imgs)//2],
        batch_imgs[-1],
    ]
else:
    selected = batch_imgs

imgs = [Image.open(p) for p in selected]
titles = [os.path.basename(p) for p in selected]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for ax, img, title in zip(axes, imgs, titles):
    ax.imshow(img)
    ax.set_title(title, fontsize=10)
    ax.axis("off")

for ax in axes[len(imgs):]:
    ax.axis("off")

plt.tight_layout()
plt.show()

### Confidence-Based Metric Curves

Bốn đường cong này được sử dụng để phân tích hành vi của mô hình theo các ngưỡng confidence khác nhau, cung cấp cái nhìn toàn diện về chất lượng dự đoán và khả năng cân bằng giữa độ chính xác và độ bao phủ.

* **Precision vs Confidence** cho thấy mức độ tin cậy của các dự đoán khi ngưỡng confidence tăng, phản ánh khả năng kiểm soát false positives của mô hình.
* **Recall vs Confidence** thể hiện khả năng phát hiện đối tượng khi thay đổi ngưỡng confidence, đặc biệt quan trọng đối với các đối tượng nhỏ và khó nhận diện như rắn.
* **Precision–Recall Curve** mô tả mối quan hệ đánh đổi giữa Precision và Recall trên toàn bộ dải ngưỡng, giúp đánh giá tổng thể hiệu năng phát hiện.
* **F1 Score vs Confidence** cung cấp tiêu chí cân bằng giữa Precision và Recall, hỗ trợ lựa chọn ngưỡng confidence tối ưu cho triển khai thực tế.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

p = Image.open(f"{HOME}/runs/detect/train/P_curve.png")
r = Image.open(f"{HOME}/runs/detect/train/R_curve.png")
pr = Image.open(f"{HOME}/runs/detect/train/PR_curve.png")
f1 = Image.open(f"{HOME}/runs/detect/train/F1_curve.png")

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

axes[0, 0].imshow(p)
axes[0, 0].set_title("Precision vs Confidence")
axes[0, 0].axis("off")

axes[0, 1].imshow(r)
axes[0, 1].set_title("Recall vs Confidence")
axes[0, 1].axis("off")

axes[1, 0].imshow(pr)
axes[1, 0].set_title("Precision–Recall Curve")
axes[1, 0].axis("off")

axes[1, 1].imshow(f1)
axes[1, 1].set_title("F1 Score vs Confidence")
axes[1, 1].axis("off")

plt.tight_layout()
plt.show()

### Dataset Validation: Label Distribution and Bounding Box Correlation

Hai biểu đồ này được sử dụng để đánh giá chất lượng và tính hợp lệ của dataset trước và sau quá trình huấn luyện mô hình.

* **Label Distribution** minh họa phân bố số lượng mẫu và kích thước bounding box của các lớp trong dataset. Biểu đồ này giúp phát hiện tình trạng mất cân bằng dữ liệu, các lớp có quá ít mẫu hoặc sự chênh lệch lớn về kích thước đối tượng.

* **Bounding Box Correlation** thể hiện mối quan hệ giữa các thuộc tính hình học của bounding box như chiều rộng, chiều cao và tỷ lệ khung hình. Phân tích này giúp phát hiện các annotation bất thường và xác nhận rằng dataset bao phủ đa dạng các kích thước và hình dạng đối tượng.

Việc trình bày đồng thời hai biểu đồ này nhằm chứng minh rằng dataset đã được kiểm tra và xác thực về mặt phân bố nhãn cũng như đặc tính hình học, đảm bảo tính phù hợp cho bài toán phát hiện rắn với đặc trưng đối tượng nhỏ, dài và lẫn nền phức tạp.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

labels = Image.open(f"{HOME}/runs/detect/train/labels.jpg")
corr = Image.open(f"{HOME}/runs/detect/train/labels_correlogram.jpg")

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(labels)
axes[0].set_title("Label Distribution")
axes[0].axis("off")

axes[1].imshow(corr)
axes[1].set_title("Bounding Box Correlation")
axes[1].axis("off")

plt.tight_layout()
plt.show()

# Test model

In [ ]:
import supervision as sv

model = YOLO(f'{HOME}/runs/detect/train/weights/best.pt')

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{DATASET_PATH}/test/images",
    annotations_directory_path=f"{DATASET_PATH}/test/labels",
    data_yaml_path=DATA_YAML_PATH
)

In [ ]:
import random

i = random.randint(0, len(ds))

image_path, image, target = ds[i]

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results).with_nms()

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

# Call tắt laptop sau khi train xong

In [ ]:
import requests

WEBHOOK_URL = "https://forcepslike-lawanda-semicalcined.ngrok-free.dev/done"  # đổi URL của bạn

try:
    r = requests.post(WEBHOOK_URL)
    print("Đã gửi tín hiệu shutdown về PC!")
except Exception as e:
    print("Không gửi được webhook:", e)